In [80]:
import pandas as pd
import ttach as tta

In [81]:
# !pip install optuna
# !pip install --upgrade numpy

In [82]:
import os
import shutil
import pathlib
import gc
from pathlib import Path
from PIL import Image
import pandas as pd
import numpy as np
import cv2 as cv
import random
import matplotlib.pyplot as plt
# from tqdm.auto import tqdm
from tqdm.notebook import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import torch
from torch.utils.data import DataLoader, random_split
from torch.utils.data import Dataset

import torchvision
from torchvision import datasets
import cv2
from torch.cuda import amp

import torchvision.transforms as T
from torchvision.transforms import Compose, ToTensor, Resize
from torchvision.utils import make_grid
import optuna

import albumentations as A
from albumentations.pytorch import ToTensorV2
import segmentation_models_pytorch as smp

In [83]:
def seed_everything(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)

In [84]:
seed_everything(42)

In [85]:
CFGS = [           
#     {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [256, 256],
#         'num_classes': 1,
#         'model_pth': '../../output/exp_01/Unet/efficientnet-b7-256/checkpoint_dice_fold0.pth',
#         'threshold': 0.34,
#         'tta': True, 
#     },
#     {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [256, 256],
#         'num_classes': 1,
#         'model_pth': '../../output/exp_01/Unet/efficientnet-b7-256/checkpoint_dice_fold1.pth',
#         'threshold': 0.29,
#         'tta': True, 
#     },
#     {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [256, 256],
#         'num_classes': 1,
#         'model_pth': '../../output/exp_01/Unet/efficientnet-b7-256/checkpoint_dice_fold2.pth',
#         'threshold': 0.4,
#         'tta': True, 
#     },
#     {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [256, 256],
#         'num_classes': 1,
#         'model_pth': '../../output/exp_01/Unet/efficientnet-b7-256/checkpoint_dice_fold3.pth',
#         'threshold': 0.38,
#         'tta': True, 
#     },
#     {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [256, 256],
#         'num_classes': 1,
#         'model_pth': '../../output/exp_01/Unet/efficientnet-b7-256/checkpoint_dice_fold4.pth',
#         'threshold': 0.41,
#         'tta': True, 
#     },
    
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [256, 256],
#         'num_classes': 1,
#         'model_pth': '../../output/exp_01_s2/Unet/efficientnet-b7-256/checkpoint_dice_fold0.pth',
#         'threshold': 0.34,
#         'tta': True, 
#     },
# #     {
# #         'model_name': 'Unet',
# #         'backbone': 'efficientnet-b7',
# #         'img_size': [256, 256],
# #         'num_classes': 1,
# #         'model_pth': '../../output/exp_01_s2/Unet/efficientnet-b7-256/checkpoint_dice_fold1.pth',
# #         'threshold': 0.29,
# #         'tta': True, 
# #     },
#     {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [256, 256],
#         'num_classes': 1,
#         'model_pth': '../../output/exp_01_s2/Unet/efficientnet-b7-256/checkpoint_dice_fold2.pth',
#         'threshold': 0.4,
#         'tta': True, 
#     },
#     {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [256, 256],
#         'num_classes': 1,
#         'model_pth': '../../output/exp_01_s2/Unet/efficientnet-b7-256/checkpoint_dice_fold3.pth',
#         'threshold': 0.38,
#         'tta': True, 
#     },
# #     {
# #         'model_name': 'Unet',
# #         'backbone': 'efficientnet-b7',
# #         'img_size': [256, 256],
# #         'num_classes': 1,
# #         'model_pth': '../../output/exp_01_s2/Unet/efficientnet-b7-256/checkpoint_dice_fold4.pth',
# #         'threshold': 0.41,
# #         'tta': True, 
# #     },
    
    
    {
        'model_name': 'Unet',
        'backbone': 'efficientnet-b7',
        'img_size': [256, 256],
        'num_classes': 1,
        'model_pth': '../../output/exp_01_s2/Unet/efficientnet-b7-256/checkpoint_dice_fold0.pth',
        'threshold': 0.34,
        'trained_by': 'rohit',
        'tta': True, 
    },
    
    
    {
        'model_name': 'Unet',
        'backbone': 'tf_efficientnet_b8',
        'img_size': [512, 512],
        'num_classes': 1,
        'model_pth': '../../output/nirjhar/effb8modelv2fold0_tf_efficientnet_b8_best_epochstage2-00.bin',
        'threshold': 0.34,
        'trained_by': 'nirjhar',
        'tta': True, 
    },
    
    
    
]

In [86]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [87]:
class ContrailDataset:
    def __init__(self, df, transform=None):
        self.df = df  
        self.images = df['image']
        self.labels = df['label']
        self.transform =transform
#         self.tta = tta_count
        
        
    def __len__(self):
        return len(self.images)
        
    def __getitem__(self, idx):
        image = np.load("../../input/" + self.images[idx]).astype(float)   
        label = np.load("../../input/" + self.labels[idx]).astype(float)
        
        
        if self.transform :
            data = self.transform(image=image, mask=label)
            image  = data['image']
            label  = data['mask']
            image = np.transpose(image, (2, 0, 1))
            label = np.transpose(label, (2, 0, 1)) 
            
        return torch.tensor(image), torch.tensor(label)

In [88]:
class Net(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        
        self.model = smp.Unet(
            encoder_name=cfg["backbone"],
            encoder_weights=None,    
            in_channels=3,            
            classes=cfg["num_classes"],     
            activation=None
        )
        
    
    def forward(self, inputs):
        mask = self.model(inputs)
        return mask
    
    
    
import segmentation_models_pytorch as smp
import timm
n_blocks = 4

class TimmSegModel(nn.Module):
    def __init__(self, cfg, segtype='unet', pretrained=True):
        super(TimmSegModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 3, stride=1, padding=1, bias=False)
        self.conv2 = nn.Conv2d(6, 12, 3, stride=1, padding=1, bias=False)
        self.conv3 = nn.Conv2d(12, 36, 3, stride=1, padding=1, bias=False)
        self.mybn1 = nn.BatchNorm2d(6)
        self.mybn2 = nn.BatchNorm2d(12)
        self.mybn3 = nn.BatchNorm2d(36)     
        self.encoder = timm.create_model(
            cfg['backbone'],
            in_chans=3,
            features_only=True,
            drop_rate=0.8,
            drop_path_rate=0.5,
            pretrained=True
        )
        self.encoder.conv_stem=nn.Conv2d(6, 72, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)

        self.encoder.blocks[5] = nn.Identity()
        self.encoder.blocks[6] = nn.Sequential(
            nn.Conv2d(self.encoder.blocks[4][2].conv_pwl.out_channels, 320, 1),
            nn.BatchNorm2d(320),
            nn.ReLU6(),
        )
        tr = torch.randn(1,6,64,64)
        g = self.encoder(tr)
        encoder_channels = [1] + [_.shape[1] for _ in g]
        decoder_channels = [256, 128, 64, 32, 16]
        if segtype == 'unet':
            self.decoder = smp.decoders.unetplusplus.decoder.UnetPlusPlusDecoder(
                encoder_channels=encoder_channels[:n_blocks+1],
                decoder_channels=decoder_channels[:n_blocks],
                n_blocks=n_blocks,
            )

        self.segmentation_head = nn.Conv2d(decoder_channels[n_blocks-1], cfg['num_classes'], kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

    def forward(self,x):
        x = F.relu6(self.mybn1(self.conv1(x)))
        global_features = [0] + self.encoder(x)[:n_blocks]
        seg_features = self.decoder(*global_features)
        seg_features = self.segmentation_head(seg_features)
        return seg_features
    
    
    
def load_model_timmunetplusplus(cfg):
    model = TimmSegModel(cfg)
    model.load_state_dict(torch.load(cfg['model_pth']))
    return model
    

In [89]:
def dice_coef(y_true, y_pred, thr=0.5, epsilon=1e-6):
    y_true = y_true.to(torch.float32)
    y_pred = (y_pred>thr).to(torch.float32)
    inter = (y_true*y_pred).sum()
    den = y_true.sum() + y_pred.sum()
    dice = ((2*inter+epsilon)/(den+epsilon)).mean()
    
    return dice



def get_transform(img_size):
    transform = A.Compose([
        A.Resize(*img_size, interpolation=cv2.INTER_NEAREST),
    ], p=1.0)
    return transform

In [90]:
# val_df = pd.read_csv("../../input/data_utils/val_df_filled.csv")
# train_df = pd.read_csv("../../input/data_utils/train_5_folds.csv")
# val_df = train_df.loc[train_df.fold == 0].reset_index(drop=True)



# final_preds = []

# for idx, cfg in enumerate(CFGS):
#     print(cfg)
#     val_transform = get_transform(cfg['img_size'])
#     valid_dataset = ContrailDataset(val_df, transform=val_transform)  

#     valid_loader = DataLoader(
#         valid_dataset, 
#         batch_size = 32, #32, 
#         shuffle = False, 
#         num_workers = 2, 
#         pin_memory = True, 
#         drop_last = False
#     )

#     model = Net(cfg)    
#     model = torch.nn.DataParallel(model).cuda()
#     model.load_state_dict(torch.load(cfg['model_pth'], map_location=torch.device('cpu'))['model'])
#     if cfg['tta']:
#         model = tta.SegmentationTTAWrapper(model, tta.aliases.flip_transform(), merge_mode='mean')

        
#     model.to(device)
#     model.eval()
    
#     preds = []
#     masks_ = []
    
#     for index, (images, masks) in enumerate(tqdm(valid_loader)):  
#         images  = images.to(device, dtype=torch.float)
#         masks  = masks.to(device, dtype=torch.float)
#         if cfg['img_size'][0] != 256:
#             masks = torch.nn.functional.interpolate(masks, size=256, mode='nearest')             
#         masks_.append(torch.squeeze(masks, dim=1))

#         with torch.inference_mode():            
#             pred = model(images)                
#             pred = pred.sigmoid()
        
#             if cfg['img_size'][0] != 256:
#                 pred = torch.nn.functional.interpolate(pred, size=256, mode='nearest') 
#             preds.append(torch.squeeze(pred, dim=1))

    
# #     model_masks = torch.stack(masks_, dim=0)
# #     model_preds = torch.stack(preds, dim=0)
    
#     model_masks = torch.cat(masks_, dim=0)
#     model_preds = torch.cat(preds, dim=0)
    
#     model_masks = torch.flatten(model_masks, start_dim=0, end_dim=1)
#     model_preds = torch.flatten(model_preds, start_dim=0, end_dim=1)  
#     dice_score = dice_coef(model_masks, model_preds).cpu().detach().numpy() 
#     dice_score_th = dice_coef(model_masks, model_preds, thr=cfg['threshold']).cpu().detach().numpy() 
        
        
        
#     best_threshold = 0.0
#     best_dice_score = 0.0
#     for threshold in [i / 100 for i in range(101)] :        
#         score = dice_coef(model_masks, model_preds, thr=threshold).cpu().detach().numpy() 
#         if score > best_dice_score:
#             best_dice_score = score
#             best_threshold = threshold
    
        
#     print(best_dice_score, best_threshold)
        
        
# #     print(dice_score, dice_score_th)
#     final_preds.append(model_preds)
    
#     del model
#     torch.cuda.empty_cache()
#     gc.collect()
    

In [91]:
val_df = pd.read_csv("../../input/data_utils/val_df_filled.csv")
train_df = pd.read_csv("../../input/data_utils/train_5_folds.csv")
val_df = train_df.loc[train_df.fold == 0].reset_index(drop=True)


final_preds = []
final_masks = []
folds = [0]

for idx, cfg in enumerate(CFGS):    
    print(idx, cfg) 
#     val_df = train_df.loc[train_df.fold == folds[idx]].reset_index(drop=True)
    val_transform = get_transform(cfg['img_size'])
    valid_dataset = ContrailDataset(val_df, transform=val_transform)  

    valid_loader = DataLoader(
        valid_dataset, 
        batch_size = 32, #32, 
        shuffle = False, 
        num_workers = 2, 
        pin_memory = True, 
        drop_last = False
    )

    
    if cfg['trained_by'] == "nirjhar":
        model = load_model_timmunetplusplus(cfg)
        if cfg['tta']: model = tta.SegmentationTTAWrapper(model, tta.aliases.hflip_transform(), merge_mode='mean')
    else:
        model = Net(cfg)    
        model = torch.nn.DataParallel(model).cuda()
        model.load_state_dict(torch.load(cfg['model_pth'], map_location=torch.device('cpu'))['model'])
        if cfg['tta']: model = tta.SegmentationTTAWrapper(model, tta.aliases.flip_transform(), merge_mode='mean')

    
    
#     if cfg['tta']:
# #         model = tta.SegmentationTTAWrapper(model, tta.aliases.flip_transform(), merge_mode='mean')
#         model = tta.SegmentationTTAWrapper(model, tta.aliases.hflip_transform(), merge_mode='mean')

        
    model.to(device)
    model.eval()
    
    preds = []
    masks_ = []
    
    for index, (images, masks) in enumerate(tqdm(valid_loader)):  
        images  = images.to(device, dtype=torch.float)
        masks  = masks.to(device, dtype=torch.float)
        masks = torch.nn.functional.interpolate(masks, size=256, mode='nearest')             
        masks_.append(torch.squeeze(masks, dim=1))

        with torch.inference_mode():        
            images = torch.nn.functional.interpolate(images,size=cfg['img_size'][0], mode='nearest')
            pred = model(images)                
            pred = torch.nn.functional.interpolate(pred.sigmoid(), size=256, mode='nearest') 
            preds.append(torch.squeeze(pred, dim=1))

    
    model_masks = torch.cat(masks_, dim=0)
    model_preds = torch.cat(preds, dim=0)
    
    
#     final_preds.append(model_preds)
#     final_masks.append(model_masks)
    
    model_masks = torch.flatten(model_masks, start_dim=0, end_dim=1)
    model_preds = torch.flatten(model_preds, start_dim=0, end_dim=1)  
    
    dice_score = dice_coef(model_masks, model_preds).cpu().detach().numpy() 
    dice_score_th = dice_coef(model_masks, model_preds, thr=cfg['threshold']).cpu().detach().numpy() 
        
        
    best_threshold = 0.0
    best_dice_score = 0.0
    for threshold in [i / 100 for i in range(101)] :        
        score = dice_coef(model_masks, model_preds, thr=threshold).cpu().detach().numpy() 
        if score > best_dice_score:
            best_dice_score = score
            best_threshold = threshold
    
        
    print(best_dice_score, best_threshold)
                
    final_preds.append(model_preds)
    
    del model
    torch.cuda.empty_cache()
    gc.collect()

0 {'model_name': 'Unet', 'backbone': 'efficientnet-b7', 'img_size': [256, 256], 'num_classes': 1, 'model_pth': '../../output/exp_01_s2/Unet/efficientnet-b7-256/checkpoint_dice_fold0.pth', 'threshold': 0.34, 'trained_by': 'rohit', 'tta': True}


  0%|          | 0/129 [00:00<?, ?it/s]

0.6962851 0.42
1 {'model_name': 'Unet', 'backbone': 'tf_efficientnet_b8', 'img_size': [512, 512], 'num_classes': 1, 'model_pth': '../../output/nirjhar/effb8modelv2fold0_tf_efficientnet_b8_best_epochstage2-00.bin', 'threshold': 0.34, 'trained_by': 'nirjhar', 'tta': True}


  0%|          | 0/129 [00:00<?, ?it/s]

0.70107603 0.79


In [92]:
# torch.save()

# f_preds =  torch.cat(final_preds, dim=0)
# f_masks =  torch.cat(final_masks, dim=0)

In [23]:
f_preds = [val.detach().cpu() for val in final_preds]
f_masks = [val.detach().cpu() for val in final_masks]

f_preds = torch.cat(f_preds, dim=0)
f_masks = torch.cat(f_masks, dim=0)

f_masks = torch.flatten(f_masks, start_dim=0, end_dim=1)
f_preds = torch.flatten(f_preds, start_dim=0, end_dim=1)  
    
print(f_masks.shape, f_preds.shape)
# torch.save(model_preds, f"../../output/probs/exp_01_s2_preds.pt")
# torch.save(model_preds, f"../../output/probs/exp_01_s2_masks.pt")
    
    

torch.Size([3153408, 256]) torch.Size([3153408, 256])


In [24]:
best_threshold = 0.0
best_dice_score = 0.0
for threshold in [i / 100 for i in range(101)] : 
    
    if threshold < 0.3 or threshold > 0.6:
        continue
    score = dice_coef(f_masks, f_preds, thr=threshold).cpu().detach().numpy() 
    print(score, threshold)
    if score > best_dice_score:
        best_dice_score = score
        best_threshold = threshold

0.6966457 0.3
0.69670683 0.31
0.6967483 0.32
0.69675624 0.33
0.69678247 0.34
0.6968062 0.35
0.69682854 0.36
0.6968573 0.37
0.6968607 0.38
0.69684297 0.39
0.69684005 0.4
0.6968414 0.41
0.69683284 0.42
0.69681466 0.43
0.6967883 0.44
0.69675183 0.45
0.69672424 0.46
0.69664913 0.47
0.69659495 0.48
0.69658124 0.49
0.6965229 0.5
0.6964799 0.51
0.6964346 0.52
0.6963845 0.53
0.69632703 0.54
0.6962536 0.55
0.6961956 0.56
0.6961283 0.57
0.69606185 0.58
0.6959818 0.59
0.6958983 0.6


In [25]:
# 0.69303155 OOF CV: 
print(best_dice_score, best_threshold)

0.6968607 0.38


In [93]:
def boost_dice(params):    
    for index, val in enumerate(params.keys()):
        if index == 0:
            preds = params[val]*final_preds[0]
        else:
            preds += params[val]*final_preds[index]
    
    
    param_sum = 0
    for key, val in params.items():
        param_sum += val

    preds = preds/param_sum
    
    
#     threshold = 0.2
#     preds1 = (nn.Sigmoid()(preds)>threshold).double()            
#     score = dice_coef(model_masks, preds1, thr=threshold).cpu().detach().numpy() # get_score(labels, preds) 
    best_threshold = 0.0
    best_score = 0.0
    for threshold in [i / 100 for i in range(101)] :
#         if threshold < 0.05 and threshold > 0.25:
#             continue
    
        preds1 = (nn.Sigmoid()(preds)>threshold).double()            
        score = dice_coef(model_masks, preds1, thr=threshold).cpu().detach().numpy() # get_score(labels, preds) 
        if score > best_score:
            best_score = score
            best_threshold = threshold
            
#     best_score = score
#     best_threshold = threshold
    return best_score, best_threshold

In [94]:
def objective(trial):
    params = {}    
    for i in range(len(final_preds)):
        params[f"w{i+1}"] = trial.suggest_float(f'w{i+1}', 0, 1)  
        
        
    score, best_threshold  = boost_dice(params)
    params['threshold'] = best_threshold
    print(params)
    return score

study = optuna.create_study(direction='maximize')
# study = optuna.create_study(direction='minimize')

study.optimize(objective, n_trials=1500)

best_params = study.best_params
best_params

[I 2023-06-22 15:17:18,249] A new study created in memory with name: no-name-9dbb33f5-2768-42ef-a30c-f9bca1691436
[I 2023-06-22 15:17:20,440] Trial 0 finished with value: 0.7030438780784607 and parameters: {'w1': 0.8516034089684172, 'w2': 0.6579994643872162}. Best is trial 0 with value: 0.7030438780784607.


{'w1': 0.8516034089684172, 'w2': 0.6579994643872162, 'threshold': 0.61}


[I 2023-06-22 15:17:22,605] Trial 1 finished with value: 0.7016289830207825 and parameters: {'w1': 0.05165751246826522, 'w2': 0.5906616946492286}. Best is trial 0 with value: 0.7030438780784607.


{'w1': 0.05165751246826522, 'w2': 0.5906616946492286, 'threshold': 0.71}


[I 2023-06-22 15:17:24,772] Trial 2 finished with value: 0.7016319632530212 and parameters: {'w1': 0.19843402528075194, 'w2': 0.12117459200983705}. Best is trial 0 with value: 0.7030438780784607.


...

{'w1': 0.8762728421160728, 'w2': 0.7863228771336537, 'threshold': 0.62}


[I 2023-06-22 16:00:29,110] Trial 1184 finished with value: 0.7031830549240112 and parameters: {'w1': 0.8401550733787041, 'w2': 0.7156833662515448}. Best is trial 495 with value: 0.7050744295120239.


{'w1': 0.8401550733787041, 'w2': 0.7156833662515448, 'threshold': 0.62}


[W 2023-06-22 16:00:31,024] Trial 1185 failed with parameters: {'w1': 0.7601440754959752, 'w2': 0.8107765958921429} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/rohits/anaconda3/envs/contrail/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_1650285/331993901.py", line 7, in objective
    score, best_threshold  = boost_dice(params)
  File "/tmp/ipykernel_1650285/298769755.py", line 26, in boost_dice
    score = dice_coef(model_masks, preds1, thr=threshold).cpu().detach().numpy() # get_score(labels, preds)
KeyboardInterrupt
[W 2023-06-22 16:00:31,025] Trial 1185 failed with value None.


KeyboardInterrupt: 

In [ ]:
best_params

In [55]:
x = torch.ones([32,3,3,256,256])

In [56]:
x.shape

torch.Size([32, 3, 3, 256, 256])

In [57]:
for a in [x[:, 0,:,:,:], x[:, 1,:,:,:]]:
    print(a.shape)

torch.Size([32, 3, 256, 256])
torch.Size([32, 3, 256, 256])


In [ ]:
pred = torch.mean(torch.stack(pred_tta), dim=0)            
